## Project 3: RNN

In [1]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [2]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('corpus length:', len(text))

606208/600901 [==============================] - 3s 5us/step
corpus length: 600901


In [3]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

total chars: 59
nb sequences: 200287


In [4]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Vectorization...
Build model...


In [5]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [7]:
# train the model, output generated text after each iteration
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()



--------------------------------------------------
Iteration 1
Epoch 1/1
200287/200287 [==============================] - 68s 338us/step - loss: 1.5504

----- diversity: 0.2
----- Generating with seed: "acrifice their old and reliable virtues,"
acrifice their old and reliable virtues, and the constitute of all the present of the present of the contrance of the constitute of the present of the most and the really and the present of the contrary and in the religious to the same the some to the most in the stands of the stract and present of the constitute of the realon of the present is not the present to the contrary to the present of the same to the stands of the lack of the s

----- diversity: 0.5
----- Generating with seed: "acrifice their old and reliable virtues,"
acrifice their old and reliable virtues, the simple to season of great with the soul the most one not the precises of a ridic prevality. in the strain the order to the origin of his wisht of the man of the their actualit

or another claimed more like-inpromplydant at they are the possible
we lave and we the "fred of
ar
and have knowledge, there morative himself. itself"--that you way and it and more as stigns, nowadays.

2ieanltic pititiesjecraciies, that man se time. as the
very
implay morality? the dacked that
obsensations of pottrences,
that,"           to a time does it who rathes to nobser who is another the . it is the kant which

----- diversity: 1.2
----- Generating with seed: "e, who at one time
or another claimed mo"
e, who at one time
or another claimed more enoogal
more appreseshednussment bradeiously: upon the "oboreablanty wouth bring and exchesturedly, the purture
if littlewoorament: indeival ekeptic bake
i es
thenguicint,sjesse all roguible , buthon not, among their well. derogers with the mind howered, strais, the evises, and kind of the two nariousness of sympathy more plata
action of the
chitasy: the christity revenge, or suffers ween do as
